# L6: Check outputs

- 사용자에게 LLM의 출력을 보여주기 전에 문제가 되는 내용은 없는지 체크하는 것이 중요

## Setup



In [ ]:
# 구글 코립일 경우
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
import os
import openai

from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)

In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens, # 모델이 출력하는 최대 토큰
    )

    return response.choices[0].message.content


### 출력물에 유해한 콘텐츠가 있는지 `Moderation`을 사용하여 확인 Check output for potentially harmful content

In [ ]:
final_response_to_customer = f"""
SmartX ProPhone은 6.1인치 디스플레이, 128GB 저장 용량, 1,200만 화소 듀얼 카메라, 5G를 갖추고 있습니다.
포토스냅 DSLR 카메라는 2420만 화소 센서, 1080p 동영상, 3인치 LCD, 교체 가능한 렌즈를 갖추고 있습니다.
55인치 디스플레이, 4K 해상도, HDR 및 스마트 TV 기능을 갖춘 CineView 4K TV를 비롯한 다양한 TV를 보유하고 있습니다.
또한 5.1채널, 1000W 출력, 무선 서브우퍼, 블루투스를 갖춘 SoundMax 홈 시어터 시스템도 갖추고 있습니다.
이러한 제품이나 다른 제품에 대해 질문이 있으신가요?
"""



response = client.moderations.create(
    input=final_response_to_customer
)

moderation_output = response.results[0]

for c, s in zip( list(moderation_output.categories), list(moderation_output.category_scores) ):
    print(f"{c[0]:>25}, {c[1]}, {s[1]:.6e}")

               harassment, False, 6.731582e-07
   harassment_threatening, False, 2.712339e-07
                     hate, False, 1.756838e-07
         hate_threatening, False, 1.285926e-07
                self_harm, False, 1.266781e-06
   self_harm_instructions, False, 7.112566e-07
         self_harm_intent, False, 7.191570e-07
                   sexual, False, 6.230678e-06
            sexual_minors, False, 3.279849e-07
                 violence, False, 3.696255e-05
         violence_graphic, False, 3.910809e-06
                self-harm, False, 1.266781e-06
            sexual/minors, False, 3.279849e-07
         hate/threatening, False, 1.285926e-07
         violence/graphic, False, 3.910809e-06
         self-harm/intent, False, 7.191570e-07
   self-harm/instructions, False, 7.112566e-07
   harassment/threatening, False, 2.712339e-07


### 제공된 제품 정보가 사실과 일치하는지 모델에게 입력해서 확인 Check if output is factually based on the provided product information

- 고객의 질의에 충실히 답변하는 경우

In [ ]:
system_message = f"""
당신은 고객 서비스 상담원이 고객 질문에 충실하게 응답하는지
그리고 상담원이 제품에 대해 인용한 모든 사실과 정보가 정확한지 검증합니다.

제품 정보와 고객 메세지, 고객 서비스 상담원 메시지는 백틱 3개, 즉 ```로 구분됩니다.

Y 또는 N 문자로 응답합니다:
Y - 출력 내용이 질문에 대한 충분한 답변을 제공하고 응답이 제품 정보를 올바르게 사용하는 경우
N - 그렇지 않은 경우

한 글자만 출력합니다.
"""

customer_message =  f"""
smartx pro phone 하고 fotosnap camera에 대해서 알려주세요. DSLR이요.
그리고 tv에 대해서 더 좀 알려주세요."""

product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""

q_a_pair = f"""
고객 메세지: ```{customer_message}```
제품 정보: ```{product_information}```
고객 서비스 상담원 메시지: ```{final_response_to_customer}```

응답이 검색된 정보를 올바르게 사용하나요?
응답이 질문에 충분히 답변하나요?

Y 또는 N 출력
"""

In [ ]:
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

Y


- 그렇지 않은 경우

In [ ]:
another_response = "인생은 초콜릿 상자와 같습니다"

q_a_pair = f"""
고객 메세지: ```{customer_message}```
제품 정보: ```{product_information}```
고객 서비스 상담원 메시지: ```{another_response}```

응답이 검색된 정보를 올바르게 사용하나요?
응답이 질문에 충분히 답변하나요?

Y 또는 N 출력
"""

messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages)
print(response)

N


- 두번째 방법은 다시 확인을 위해 LLM을 호출해야 하므로 비용이 크게 증가

- 매우 민감한 정보를 다루는 경우에 시도해봄직 함

- 체크 결과를 신뢰할 수 있어야 하므로 응답 LLM보다 더 성능이 좋고 비싼 LLM에게 체크를 맡기는 것이 좋음

- 다른 방법으로 일종의 앙상블 방식을 사용하는 것으로 다양한 LLM에게 질의하고 응답을 모두 모아 성능이 뛰어난 LLM에게 응답을 고르게 하는 방식도 가능